<a href="https://colab.research.google.com/github/nid-22/Langchain_Multitool_Retail_Agent/blob/main/Langchain_multitool_retail_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!install openai
!pip install python-dotenv

install: missing destination file operand after 'openai'
Try 'install --help' for more information.


In [9]:
import os
# os.environ['OPENAI_API_KEY'] =

In [11]:
from openai import OpenAI
import openai
import os
from dotenv import load_dotenv
load_dotenv()

llm_model = 'gpt-3.5-turbo'
client = OpenAI()

How to Call OpenAI APIs

In [12]:
from tempfile import TemporaryDirectory
def chat_completion(prompt, model=llm_model):
  messages = [
    {"role": "user", "content": prompt}
 ]
  response = client.chat.completions.create(
      model = model,
      messages = messages
  )
  return response.choices[0].message.content

prompt = 'What is 1+1?'

chat_completion(prompt)

'1+1 equals 2.'

In [13]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

chat_completion(prompt)

"Arrgh, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, buddy!"

The customer emails can be in any language, we need to convert them into this format
style = American English in a calm and respectful tone
Lets use Langchain for this

In [ ]:
!pip install --upgrade Langchain
!pip install -U langchain-openai


## Use Langchain

In [15]:
from langchain_openai import ChatOpenAI

In [16]:
llm = ChatOpenAI(model = llm_model, temperature=0.0)

##   Prompt Template

Prompt templates let us standardize, reuse, and parameterize prompts so the LLM receives consistent instructions while allowing dynamic inputs.

Key points:

Avoid hardcoding prompts

Inject variables safely

Improve consistency across calls

Easier maintenance and experimentation

Enables chaining, agents, and tools

In [17]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'), additional_kwargs={})])

In [18]:
prompt_template.input_variables

['style', 'text']

In [19]:
customer_msg = prompt_template.format_messages(style = style, text = customer_email)

In [20]:
print(type(customer_msg))
print(type(customer_msg[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


## Call the llm

In [22]:
response = llm.invoke(customer_msg)

In [23]:
response.content

"Oh man, I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

## Try with different style

In [21]:
style_2 = """Hindi \
in a calm and respectful tone
"""

customer_msg_2 = prompt_template.format_messages(style = style_2, text = customer_email)

response = llm.invoke(customer_msg_2)

response.content

'मुझे बहुत गुस्सा आ रहा है कि मेरे ब्लेंडर का ढक्कन उड़ गया और मेरे रसोई की दीवारों पर स्मूथी से छिड़क गया! और और बुरी बात यह है कि वारंटी मेरे रसोई को साफ करने का खर्च नहीं कवर करती। मुझे अभी तुम्हारी मदद चाहिए, मित्र!'

# OUTPUT PARSER

Output parsers enforce structured, predictable outputs from LLMs so the response can be reliably used by downstream code.

Key points :

LLMs return free-form text by default

Parsers convert text → JSON / schema

Prevents brittle string parsing

Enables automation, validation, and safety

Critical for production systems

In [22]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [23]:
prompt_template = ChatPromptTemplate.from_template(review_template)

msg = prompt_template.format_messages(text = customer_review)

response = llm.invoke(msg)

response.content

'{\n    "gift": true,\n    "delivery_days": 2,\n    "price_value": ["It\'s slightly more expensive than the other leaf blowers out there"]\n}'

In [25]:
from pydantic import BaseModel, Field
from typing import List

class ReviewAnalysis(BaseModel):
    gift: bool = Field(
        description="Was the item purchased as a gift for someone else?"
    )
    delivery_days: int = Field(
        description="Number of days it took to arrive. Use -1 if unknown."
    )
    price_value: List[str] = Field(
        description="Sentences mentioning value or price."
    )

from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=ReviewAnalysis)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"gift": {"description": "Was the item purchased as a gift for someone else?", "title": "Gift", "type": "boolean"}, "delivery_days": {"description": "Number of days it took to arrive. Use -1 if unknown.", "title": "Delivery Days", "type": "integer"}, "price_value": {"description": "Sentences mentioning value or price.", "items": {"type": "string"}, "title": "Price Value", "type": "array"}}, "required": ["gift", "delivery_days", "price_value"]}
```


In [26]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(
    template=review_template_2
)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

response = llm.invoke(messages)
response.content

'{\n  "gift": false,\n  "delivery_days": 2,\n  "price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}'

In [27]:
parsed_output = output_parser.parse(response.content)

In [28]:
parsed_dict = parsed_output.model_dump()
print(parsed_dict)


{'gift': False, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


## Memory

• LLMs are statelessa and do not remember past interactions by default.
• Each request must include any required context.
• Sending the entire chat history increases:
  – Token cost
  – Latency
  – Risk of hitting context window limits

## Types of memory in Langchain
1) ConversationBufferMemory
• Stores the entire conversation history
• Sends all previous messages with every prompt
• Token usage grows unbounded
• Not suitable for long conversations or production
--------------------------------------------------
2) ConversationBufferWindowMemory
• Stores only the last K interactions (window size)
• Drops older messages automatically
• Older context is lost
--------------------------------------------------
3) ConversationTokenBufferMemory
• Maintains history up to a maximum token limit
• Removes oldest messages when token limit is exceeded
• Older context is discarded
• No semantic compression
--------------------------------------------------
4) ConversationSummaryBufferMemory
• Summarizes older conversation into a compact summary
• Uses an LLM to update the summary
• Preserves long-term context
• Controls token growth
• Long-running conversations
• Production chat assistants

# PROJECT STARTS HERE

We have a products catalogue and we build an agent which will answer queries using various tools

In [29]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Colab Notebooks/langchain/OutdoorClothingCatalog_1000.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain-community
!pip install docarray

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai import OpenAIEmbeddings

loader = CSVLoader(file_path=file_path)
documents = loader.load()

In [37]:
print(len(documents))
print(documents[0])

1000
page_content=': 0
name: Women's Campside Oxfords
description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. 

Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. 

Specs: Approx. weight: 1 lb.1 oz. per pair. 

Construction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. 

Questions? Please contact us for any inquiries.' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/langchain/OutdoorClothingCatalog_1000.csv', 'row': 0}


## Store embedded documents into vectorstore

In [30]:
embeddings = OpenAIEmbeddings()

vectorstore = DocArrayInMemorySearch.from_documents(
    documents,
    embedding=embeddings
)

In [31]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

results = vectorstore.similarity_search(query, k=3)

for doc in results:
    print(doc.page_content)

: 618
name: Men's Tropical Plaid Short-Sleeve Shirt
description: Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. With a traditional fit that is relaxed through the chest, sleeve, and waist, this fabric is made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets, this shirt is imported and provides the highest rated sun protection possible. 

Sun Protection That Won't Wear Off. Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays.
: 374
name: Men's Plaid Tropic Shirt, Short-Sleeve
description: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and 

## Create a retriever

In [34]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


## Create a Chain

In [35]:
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""
Use the following context to answer the question.
If you don't know, say you don't know.

Context:
{context}

Question:
{question}
""")

chain = (
    {"context": retriever, "question": lambda x: x}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("Which products are good for cold and wet weather?")

"The products that are good for cold and wet weather are the Women's Rain Seekers, Lace-Up Boot and the Swix XC Comfort Gloves."

# CREATING AGENTS

## What we require our agent to do

Tools we’ll build:

-  Product Search Tool (semantic search)

-  Natural Language Filter Tool (structured filtering)

-  Outfit Builder Tool (multi-step planner)

- Comparison Tool (compare products)


The agent will:

User

 ↓

Agent (LLM + memory)

 ↓ decides

Tools:
  - ProductSearchTool (semantic search)
  - FilterTool (natural language → structured filters)
  - CompareTool (compare products)
  - OutfitBuilderTool (multi-step planning)

 ↓

Final Answer (grounded in catalog)

In [ ]:
!pip install pandas tabulate

In [51]:
import pandas as pd
df = pd.read_csv(file_path)

### Tool 1: Product Recommendation (semantic search)

In [52]:
from langchain.tools import tool

@tool
def recommend_products(query: str) -> str:
    """Recommend relevant products based on a natural language query."""
    docs = retriever.invoke(query)
    return "\n\n".join([d.page_content for d in docs])


### Tool 2: Product Comparison (outputs a table)

In [53]:
from tabulate import tabulate

@tool
def compare_products(product_names: str) -> str:
    """
    Compare products given a comma-separated list of product names.
    Outputs a comparison table.
    """
    names = [n.strip().lower() for n in product_names.split(",")]

    rows = df[df["name"].str.lower().isin(names)]

    if rows.empty:
        return "No matching products found."

    return tabulate(
        rows[["name", "description"]],
        headers="keys",
        tablefmt="github"
    )


Tool 3: Outfit Builder (gender-aware)

In [54]:
@tool
def build_outfit(gender: str, weather: str) -> str:
    """
    Build an outfit with topwear, bottomwear, and shoes based on gender and weather.
    """
    gender = gender.lower()

    def pick(query):
        docs = retriever.invoke(query)
        return docs[0].page_content if docs else "Not found"

    top = pick(f"{gender} {weather} jacket or topwear")
    bottom = pick(f"{gender} {weather} pants or trousers")
    shoes = pick(f"{gender} {weather} shoes")

    return f"""
TOPWEAR:
{top}

BOTTOMWEAR:
{bottom}

SHOES:
{shoes}
"""


Tool 4: Natural Language Filter (pandas)

In [55]:
@tool
def filter_products(filter_query: str) -> str:
    """
    Filter products using natural language (e.g., 'women shoes under $100').
    Uses pandas internally.
    """
    q = filter_query.lower()

    filtered = df.copy()

    if "women" in q:
        filtered = filtered[filtered["name"].str.contains("women", case=False)]
    if "men" in q:
        filtered = filtered[filtered["name"].str.contains("men", case=False)]

    if "shoe" in q:
        filtered = filtered[filtered["description"].str.contains("shoe", case=False)]

    return filtered.head(10).to_string(index=False)


Error handling middleware for tools

In [59]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Tool error: {str(e)}",
            tool_call_id=request.tool_call["id"]
        )


Create the Agent

In [60]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o",
    tools=[
        recommend_products,
        compare_products,
        build_outfit,
        filter_products
    ],
    system_prompt=(
        "You are a smart shopping assistant for a clothing catalog. "
        "Choose the best tool based on the user's request. "
        "Use tables when comparing products. "
        "Be concise and accurate."
    ),
    middleware=[handle_tool_errors]
)


In [63]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Recommend waterproof jackets for cold weather"}
    ]
})
final_answer = result["messages"][-1].content
print(final_answer)


Here are some recommended waterproof jackets ideal for cold weather:

1. **Outdoor Adventurer Rain Shell**
   - Features: Durable laminate interior, TEK waterproof technology, adjustable hood/cuffs/hem, packs into its own pocket.
   - Additional Info: Made from 100% recycled nylon, breathable comfort.
   - Care: Machine wash and dry.
   - Fit: Slightly fitted, falls below hip.

2. **Girls' Summit Waterproof Insulated 3-in-1 Jacket**
   - Features: 3-in-1 design, PrimaLoft Silver insulation, waterproof and breathable shell, media ports.
   - Additional Info: Slightly fitted, offers three wear options for various weather conditions.
   - Temperature Rating: Light activity: to 20°, Moderate activity: to -5°.
   - Care: Machine wash and dry.

3. **Women's Waterproof Primaloft Packable Jacket**
   - Features: PrimaLoft insulation with Aerogel technology, TEK 2L waterproof fabric, adjustable hood/cuffs/hem, pit zips.
   - Additional Info: Offers superior warmth and protection, easy-to-pack d

In [67]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Compare Women's water proof hiking shoes, Men's lace up shoes"}
    ]
})
final_answer = result["messages"][-1].content
print(final_answer)

Here is a comparison of some waterproof hiking shoes for women and men:

| Name                                                    | Gender | Weight/Pair    | Upper Material                     | Waterproofing       | Special Features                                                                                     |
|---------------------------------------------------------|--------|-----------------|-----------------------------------|---------------------|------------------------------------------------------------------------------------------------------|
| Men's Trail Model 4 All-Terrain Waterproof Hiking Shoes | Men    | Approx. 2 lb. 5 oz. | Suede and fabric                  | TEK2.5 barrier      | VertiGrip outsole, cushioned EVA midsole, heel-and-toe bumpers                                       |
| Women's Merrell Moab 2 Waterproof Trail Shoes           | Women  | Approx. 13 oz.  | Performance suede leather-and-mesh | Waterproof layer    | Ventilated tongue, rubber toe cap

In [70]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Build an outfit for a woman for cold rainy weather"}
    ]
})

final_answer = result["messages"][-1].content
print(final_answer)


Here's an outfit for a woman suitable for cold rainy weather:

### Topwear
- **Name**: Girls' Summit Waterproof Insulated 3-in-1 Jacket
- **Description**: This versatile jacket offers three options: a waterproof shell, a warm insulated liner, or both together for full protection. Made of nylon ripstop with PrimaLoft Silver insulation, it's breathable, comfortable, and perfect for temperatures as low as -5° during moderate activity.
- **Features**: Media port, fully taped seams, machine washable.

### Bottomwear
- **Name**: Women's Waterproof Hiking Trousers
- **Description**: Tough and durable waterproof rain pants with a laminate interior for rugged protection and breathable comfort. Designed with an adjustable waist and zippered pockets for practicality.
- **Fabric**: Made from 100% recycled nylon and features TEK waterproof technology.

### Shoes
- **Name**: Women's Rain Seekers, Lace-Up Boot
- **Description**: These boots offer the waterproof protection of rubber boots combined wit

In [71]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Show me women's shoes"}
    ]
})

final_answer = result["messages"][-1].content
print(final_answer)


Here are some women's shoes available in the catalog:

| Product Name                                        | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|-----------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Improvements to be done:

- add structured JSON schemas to tools
- Add conversation memory